In [1]:
import geemap
import ee

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
import os
os.chdir(r'D:\Work\Problem')

In [7]:
import geopandas as gpd

shp_file = r'JX.shp'
shp_gdf = gpd.read_file(shp_file)
shp = geemap.gdf_to_ee(shp_gdf)

In [8]:
roi = shp.first().geometry()
Map.addLayer(roi, {}, 'roi')
Map.center_object(roi, 11)

In [9]:
def apply_scale_factors(image):
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True).updateMask(qa_mask)


In [7]:
images = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).filterDate('2022', '2023')\
            .map(apply_scale_factors).select('SR_B[2-4]')

In [ ]:
images.first().getInfo()

In [12]:
Map.addLayer(images.median(), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min':0, 'max':0.3}, "imags")

In [ ]:
image = images.mosaic()

In [8]:
geemap.download_ee_image(images.median().clip(roi).multiply(1000), 'landsat.tif', scale=30, crs='epsg:3857', region=roi)

landsat.tif: |                                                         | 0.00/199M (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None


In [14]:
geemap.download_ee_image(images.mosaic().clip(roi).multiply(1000).select("SR_B4"), 'landsat_B4.tif', scale=30, crs='epsg:3857', region=roi)

landsat_B4.tif: |                                                     | 0.00/66.2M (raw) [  0.0%] in 00:00 (et…

In [ ]:
image = ee.Image('LANDSAT/LT05/C01/T1/LT05_121040_19871217')
# Map.addLayer(image, {'bands': ['B4', 'B3', 'B2']}, "imag")

In [ ]:
roi = Map.draw_features[1].geometry()

In [ ]:
geemap.ee_export_image_to_drive(image, description='LE7_121040_20000127',
                                folder='Image', scale=30)

In [ ]:
file = r"E:/LT05_121040_19871217.tif"
geemap.ee_export_image(image, region=roi, filename=file, scale=15, file_per_band=False)

In [ ]:
Map.draw_features